近期公司内部，经常会因为客户的紧急需求，需要拉取全部设备的信息。
在我们的配置存储格式中，会有许许多多的config，如果每次都需要自己手动去整理的话，非常的麻烦。所以我希望能够将它们的一些信息整理成表格列出来。

思路：
1.读取目录下的所有配置文件名，例 20180128 shcesw2A 与 20180129 shcesw2A；
我希望通过统一的命名格式，通过**设备名**，确认同设备，再通过**日期**来判断配置新旧。进行信息的筛选。

2.通过配置筛选出各个厂商对应的关键信息，具体如下：
设备地点，楼层，房间，设备型号，环境，主机名，厂商EOL，SN，后续型号，L3链接信息；

3.将信息筛选完毕后，逐一输出；

In [ ]:
import os
import numpy as np
import textfsm
import csv
def get_path(path):
    #申明全局
    global all_path
    for root, dirs, files in os.walk(path):
        for filepath in files:
            #print(os.path.join(root, filepath))
            all_path.append(os.path.join(root, filepath))
            
if __name__ == "__main__":
    path = "O:\py" 
all_path=[]
get_path(path)
all_path_str = ','.join(all_path)
print(all_path_str)

In [ ]:
def export(now):   
    #筛选所有的设备list
    global all_device
    global all_device_date
    global all_device_device
    link_start=now.find('20')
    link_end=now.find('_',link_start)
   
    name_start=now.find('_',link_start-1)
    name_end=now.find('.txt',name_start)
    
    if now[link_start:link_end].startswith("20"): #创建判断条件,如果在函数now(之后调用的page_hero)中,包括"http"
        #print(now[link_start:link_end]+now[name_start+1:name_end])           #则输出1
        print(now[link_start:link_end])
        print(now[name_start+1:name_end])        #输出2
        all_device_date.append(now[link_start:link_end])
        all_device_device.append(now[name_start+1:name_end])
        all_device.append(now[link_start:link_end]+now[name_start:name_end])
        return name_end                          #返回值为 name_end (新的起始位置)
    else:
        return -1    
label=0
path_t = []
all_device = []
all_device_date = [] 
all_device_device = []
nput_text = 0
row_all = []
while True:
    all_path_str=all_path_str[label:]
    label=export(all_path_str)
    if label == -1:
            break;

In [201]:
#踢出old的配置文件
'''for i in range(0,len(all_device)):
    print("*",all_device)
    print("*",all_device_date)
    print("*",all_device_device)
    while i != len(all_device) and all_device_device[i] == all_device_device[i+1]:
        if all_device_date[i] >= all_device_date[i+1]:
            all_device_device.pop[i+1]
        else:
            all_device_device.pop[i]
    else:
        pass
    print(all_device_device)'''


'for i in range(0,len(all_device)):\n    print("*",all_device)\n    print("*",all_device_date)\n    print("*",all_device_device)\n    while i != len(all_device) and all_device_device[i] == all_device_device[i+1]:\n        if all_device_date[i] >= all_device_date[i+1]:\n            all_device_device.pop[i+1]\n        else:\n            all_device_device.pop[i]\n    else:\n        pass\n    print(all_device_device)'

In [ ]:
#print(all_device)
#print(all_path_str)
#筛选出对应config的对应path
#all_path = all_path.split(",")
#print(all_path)
#all_path_ = all_path.replace("\\", "\")


for i in range (0,len(all_device)):    
    for j in range (0,len(all_path)):
        if all_device[i] in all_path[j]:
            path_t.append(all_path[j])
        else:
            pass
print(path_t)
 

In [203]:
def config(now):   
    global input_text
    #筛选出需要的配置
    hostname_start=now.find('hostname')
    hostname_end=now.find('!',hostname_start)
   
    version_start=now.find('Version')
    version_end=now.find(', ',version_start)
    
    sn_start=now.find('sn')
    sn_end=now.find('!',sn_start)    
        
    
    if now[hostname_start:hostname_end].startswith("hostname"): #创建判断条件,如果在函数now(之后调用的page_hero)中,包括"http"
        input_text=(now[hostname_start+9:hostname_end-1]+now[version_start+8:version_end]+now[sn_start+3:sn_start+14])
        #print(now[hostname_start+9:hostname_end-1])
        #print(now[version_start+8:version_end])
        #print(now[sn_start+3:sn_start+14])
        
    else:
        return -1  

while True:
    config(line)
    #print(input_text)
    if label == -1:
         break;
        

In [ ]:
#根据目录读取txt
out = open('Z:\\common\\006 Lab\\NetDevOps\\13网络设备的配置管理\\export_config.csv','w')#这种方式会覆盖之前的内容。
csv_writer = csv.writer(out)
         
        #print(row_all)
for i in range(0,len(path_t)):
    global row_all
    TEMP_FILE="Z:\\common\\006 Lab\\NetDevOps\\13网络设备的配置管理\\textFSM.cfg" 
    INPUT_FILE=str(path_t[i])
    #打开模版文件，并且初始化FSM类
    fsm = textfsm.TextFSM(open(TEMP_FILE))
    #读取需要处理的文本，需要读取文件内容，而且不是文件句柄。
    input_text = open(INPUT_FILE).read()
    #使用parseText方式解析文件
    fsm_results = fsm.ParseText(input_text)
    #print(fsm.header)
    for row in fsm_results:
        print(row)         
        csv_writer.writerow(row)
        
out.close()